<img src="../logo.png" height="200" width="900"> 

# <center> Сбор данных: грязная работа руками API </center>

__API (Application Programming Interface__ — это уже готовый код, который можно всунуть в свой код! Многие сервисы, в том числе Google и Вконтакте, предоставляют свои уже готовые решения для вашей разработки.

Примеры: 

* [Контактовский API](https://vk.com/dev/methods)
* [API twitter](https://developer.twitter.com/en/docs.html) 
* [API youtube](https://developers.google.com/youtube/v3/)
* [API google maps](https://developers.google.com/maps/documentation/) 
* [Aviasales](https://www.aviasales.ru/API)
* [Yandex Translate](https://yandex.ru/dev/translate/)

Оно есть почти везде! Мы сейчас с вами обсудим как работать с таким API на примере социальной сетки вконтакте. 

# 1. Доступ к API vk

Зачем может понадобиться доступ к API контакта, думаю, объяснять не надо. Социальная сетка — это тонны различной полезной информации, которую можно заиспользовать для своего рисёрча. [В документации](https://vk.com/dev/manuals) очень подробно описано как можно работать с API контакта и к чему это приводит. 

Но для начала к API нужно получить доступ. Для этого придётся пройти пару бюрократических процедур (о, боже, эти два предложения были так бюрократически сформулированы, что мне захотелось отстоять в очереди).

Первая такая процедура заключается в создании своего приложения. Для этого переходим по [ссылке](http://vk.com/editapp?act=create) и проходимся по необходимым шагам:

<img align="center" src="pictures/app_creation_1.png" width="500">

После подтверждения своей личности по номеру телефона, попадаем на страницу свежесозданного приложения

<img align="center" src="pictures/app_creation_2.png" width="500">

Слева нам будем доступна вкладка с настройками, перейдя в неё мы увидим все необходимые нам для работы с приложением параметры:
<img align="center" src="pictures/app_creation_3.png" width="500">

Отсюда в качестве токена можно забрать сервисный ключ доступа. Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Иногда нужны дополнительные доступы. Для того, чтобы получить их, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=**********&scope=8198&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

В итоге по этому запросу будет сформирована ссылка следующего вида:
> https://oauth.vk.com/blank.html#access_token=25b636116ef40e0718fe4d9f382544fc28&expires_in=86400&user_id=*******

Первый набор знаков — `access token`, т.е. маркер доступа. Вторая цифра (`expires_in=`) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (`user_id=`) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа. Для удобства сохраним его в отдельном файле или экспортируем в глобальную область видимости. В целях безопасности ваших данных не стоит нигде светить токенами и тем более выкладывать их в открытый доступ. __Так можно и аккаунта случайно лишиться.__ Берегите токен смолоду. 

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр `scope=8198.` Это мы просим доступ к конкретным разделам. Подробнее познакомиться с взаимно-однозначным соответствием между числами и правами можно [в документации.](https://vk.com/dev/permissions) Например, если мы хотим получить доступ к друзьям, фото и стенам, мы подставим в scope цифру 2+4++8192=8198.

In [1]:
# мой номер странички
myid = ' '  # вставить номер странички

# версия используемого API
version = '5.103' 

# подгружаем токен из файлика на компьютере
with open('token.txt') as f:
    token = f.read()

In [2]:
import requests
from tqdm import tqdm_notebook

In [4]:
method = "users.get"
parameters = "user_ids=6045249"

url = 'https://api.vk.com/method/' + method + \
'?' + parameters + '&v=' + version + '&access_token=' + token

In [6]:
response = requests.get(url)
response.json()

{'response': [{'id': 6045249,
   'first_name': 'Филипп',
   'last_name': 'Ульянкин',
   'is_closed': False,
   'can_access_closed': True}]}

In [8]:
response.json()['response'][0]['first_name']

'Филипп'

In [9]:
def vk_download(method, parameters):
    url = 'https://api.vk.com/method/' + method + \
'?' + parameters + '&v=' + version + '&access_token=' + token
    
    response = requests.get(url)
    return response.json()['response']

In [11]:
vk_download('users.get','user_ids=6045249,1&fields=bdate,city')

[{'id': 6045249,
  'first_name': 'Филипп',
  'last_name': 'Ульянкин',
  'is_closed': False,
  'can_access_closed': True,
  'bdate': '3.10.1994',
  'city': {'id': 1, 'title': 'Москва'}},
 {'id': 1,
  'first_name': 'Павел',
  'last_name': 'Дуров',
  'is_closed': False,
  'can_access_closed': True,
  'bdate': '10.10.1984',
  'city': {'id': 2, 'title': 'Санкт-Петербург'}}]

# Почиташки

* [Немного устаревший туториал по vk API](https://nbviewer.jupyter.org/github/FUlyankin/ekanam_grand_research/blob/master/0.%20vk_parser_tutorial.ipynb)

<img src="pictures/cats.jpg" height="200" width="1200">  